In [1]:
###### Config #####
import sys, os, platform
if os.path.isdir("ds-assets"):
  !cd ds-assets && git pull
else:
  !git clone https://github.com/lutzhamel/ds-assets.git
colab = True if 'google.colab' in os.sys.modules else False
system = platform.system() # "Windows", "Linux", "Darwin"
home = "ds-assets/assets/"
sys.path.append(home)

Already up to date.


In [2]:
# notebook level imports
import sys
import pandas
import dsutils
from sklearn.metrics.pairwise import euclidean_distances

# Natural Language Processing (NLP)

Some of the most important data in our society is represented as unstructured text:

* Medical records
* Court cases
* Insurance documents

Other data perhaps not as fundamental but that provides interesting insights into trends and mindsets:

* Twitter/X and other online blogs
* News feeds


In all of these cases we want to extract meaning from the unstructured text:

* Perhaps we want to do classification (medical records - high risk/low risk)
* Perhaps we want to do a topic analysis of the twitter/X feeds
* Perhaps we would like to construct a recommendation engine for news feeds

Regardless, what the task, we need to convert the unstructured text into something that we can work with and perhaps most importantly, our models can work with.

☞ The **Vector Model** of text (sometimes called the **Bag-of-Words model**)


## The Vector Model

The vector model converts a document with unstructured text into a **point in an n-dimensional coordinate system** where the coordinate system is defined by the words contained in the text.

Consider: the quick brown fox jumps over the lazy dog

This text can be represented as the tuple rearranged in alphabetical order,
```
(brown,dog,fox,jumps,lazy,over,quick,the)
```

Let’s consider the fact that we have multiple documents and represent them as tuples,

* Doc 1: the quick brown fox jumps over the lazy dog &rarr; `(brown,dog,fox,jumps,lazy,over,quick,the)`
* Doc 2: rudi is a lazy brown dog &rarr; `(a,brown,dog,is,lazy,rudi)`
* Doc 3: princess jumps over the dog &rarr; `(dog,jumps,over,princess,the)`


In order to compare the documents we create a tuple of the **union** of the words appearing in the
sentence tuples,
```
(a,brown,dog,fox,is,jumps,lazy,over,princess,quick,rudi,the)
```
and represent each document as bit vectors with the same length as the tuple above and with 1's and 0's
indicating if the document contains a word at a particular tuple position or not,

* Doc 1: the quick brown fox jumps over the lazy dog &rarr; `(0,1,1,1,0,1,1,1,0,1,0,1)`
* Doc 2: rudi is a lazy brown dog &rarr; `(1,1,1,0,1,0,1,0,0,0,1,0)`
* Doc 3: princess jumps over the dog &rarr; `(0,0,1,0,0,1,1,1,1,0,0,1)`

Notice that our word tuple now has become our coordinate system, in this case with 12 dimensions, and each document is now a point in this 11-dimensional space.  Something like the following where point $P = (x,y,z)$ but in 12 dimensions,

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Rectangular_coordinates.svg/1280px-Rectangular_coordinates.svg.png" width="350" height="300">

> The nice thing about this vector model representation is that **we can do mathematics on documents!**



Given our vector model of the three docs we can ask questions like this,

> Is doc2 or doc3 more similar to doc1?

Since all three documents are considered points in our coordinate system we can Euclidean distances in that coordinate system to answer that question. More specifically, we can answer this question by considering the Euclidean distances doc1 &harr; doc2 and doc1 &harr; doc3 in our coordinate system.  

The Euclidean distance d in n-dimensional space between two points $p$ and $q$ is defined as:

$d(p,q) = \sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\ldots+(p_n-q_n)^2}$

In our case the point $p$ and $q$ are document vectors and $p_i$ and $q_i$ are the components of the respective
vectors.

In order to answer our question we have to perform the following computations,

* $d(doc1, doc2) = \sqrt{(0-1)^2+(1-1)^2+(1-1)^2+(1-0)^2+(0-1)^2+(1-0)^2+(1-1)^2+(1-0)^2+(0-0)^2+(1-0)^2+(0-1)^2+(1-0)^2}                      = \sqrt{1+0+0+1+1+1+0+1+0+1+1+1} = \sqrt{8} = 2.83$

* $d(doc1,doc3) = \sqrt{(0-0)^2+(1-0)^2+(1-1)^2+(1-0)^2+(0-0)^2+(1-1)^2+(1-1)^2+(1-1)^2+(0-1)^2+(1-0)^2+(0-0)^2+(1-1)^2} = \sqrt{0+1+0+1+0+0+0+0+1+1+0+0} = \sqrt{4} = 2.0$

> So, doc3 is more similar to doc1 than doc2!


## The Vector Model in Sklearn - The DocTerm Matrix

Let's try the above in sklearn. First we construct our vector model, traditionally called the **docterm matrix**.

In [3]:
# set up our documents
doc_names = ["doc1", "doc2", "doc3"]
docs = ["the quick brown fox jumps over the lazy dog",
        "rudi is a lazy brown dog",
        "princess jumps over the lazy dog"]

In [4]:
# build the docterm matrix
docterm = dsutils.docterm_matrix(docs, doc_names)
docterm

,a,brown,dog,fox,is,jumps,lazy,over,princess,quick,rudi,the
doc1,0,1,1,1,0,1,1,1,0,1,0,1
doc2,1,1,1,0,1,0,1,0,0,0,1,0
doc3,0,0,1,0,0,1,1,1,1,0,0,1


Now we are ready to answer the question from above which doc is more similar to doc1.

In [5]:
# print pairwise distances between documents
distances = pandas.DataFrame(data=euclidean_distances(docterm),
                             index=doc_names,
                             columns=doc_names)
distances

,doc1,doc2,doc3
doc1,0.000000,2.828427,2.000000
doc2,2.828427,0.000000,2.828427
doc3,2.000000,2.828427,0.000000


> Just as we computed by hand - doc3 is more similar to doc1 than doc2.

# Real World Data: News Articles

The data set we will be using are articles from a newsgroup feed (think chat room before chat rooms existed).

We will look at two newsgroups:
* Politics
* Space


In [6]:
# get the newsgroup data
newsgroups = pandas.read_csv(home+"newsgroups.csv")

# create doc names vector in order to easily identify docs in out dataframes
# we make the doc names our index in the newsgroups dataframe.
doc_names = [f'doc{i}' for i in range(newsgroups.shape[0])]
newsgroups.index = doc_names
newsgroups.head(n=10)

,text,label
doc0,From: demon@desire.wright.edu (Not a Boomer)\n...,space
doc1,From: dreitman@oregon.uoregon.edu (Daniel R. R...,space
doc2,From: mcgoy@unicorn.acs.ttu.edu (David McGaugh...,space
doc3,From: blh@uiboise.idbsu.edu (Broward L. Horne)...,space
doc4,From: wiggins@cecer.army.mil (Don Wiggins)\nSu...,space
doc5,From: nickh@CS.CMU.EDU (Nick Haines)\nSubject:...,politics
doc6,From: mike@gordian.com (Michael A. Thomas)\nSu...,space
doc7,From: jbreed@doink.b23b.ingr.com (James B. Ree...,politics
doc8,From: baalke@kelvin.jpl.nasa.gov (Ron Baalke)\...,politics
doc9,From: DPierce@world.std.com (Richard D Pierce)...,politics


In [7]:
# overall shape of the data
newsgroups.shape

(1058, 2)

In [8]:
# a typical article
print(newsgroups['text']['doc1'])

From: dreitman@oregon.uoregon.edu (Daniel R. Reitman, Attorney to Be)
Subject: Re: Traffic Case
Article-I.D.: oregon.5APR199315572465
Distribution: world
Organization: University of Oregon
Lines: 20
NNTP-Posting-Host: oregon.uoregon.edu
News-Software: VAX/VMS VNEWS 1.41

In article <1993Apr5.140934.876@colorado.edu>,
 ajteel@dendrite.cs.Colorado.EDU (A.J. Teel) writes...
>	The [McDonald] case was dismissed in the interests of Justice

On whose authority do you have this and on what grounds was it 
dismissed?

						Daniel Reitman

HOW NOT TO WRITE A DEED

One case involved the construction of a conveyance to grantees "jointly, as 
tenants in common, with equal rights and interest in said land, and to the 
survivor thereof, in fee simple. . . . To Have and to Hold the same unto the 
said parties hereto, equally, jointly, as tenants in common, with equal rights 
and interest for the period or term of their lives, and to the survivor thereof 
at the death of the other."

The court held th

In [9]:
# good representation of both kinds of articles?
newsgroups['label'].value_counts()

label
politics    593
space       465
Name: count, dtype: int64

## The Docterm Matrix

We construct the docterm matrix. 

In [10]:
# build vector model of the newsgroup data      
docterm = dsutils.docterm_matrix(newsgroups['text'], doc_names)   
docterm

,0,00,000,0000,00000,000000,000007,000021,000062david42,00041032,...,zowie,zullen,zulu,zumwalt,zurbrin,zwaartepunten,zwak,zwakke,zware,zwarte
doc0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc1053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1054,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1055,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Look at at the shape of the docterm matrix, we see that we have about 23,000+ different features.  That means our newsgroup articles exist in a 23,000+ dimensional space. When we look at the features it is clear that there are many "nonsense" features.  We need more filtering!

## Minimum Doc Frequency and Regular Expressions

From this it is clear that we want to do some additional filtering:
* Minimum doc frequency = 2 -- that is, any word has to appear at least twice in the document collection
* Delete anything that is not a word - get rid of things like ‘000’ etc., we use the token pattern arg for that.


In [11]:
# build vector model with min_df=2 and token_pattern="[a-zA-Z]+"
# this means that we only consider words (no punctuation) and
# we only consider words that appear at least twice     
docterm = dsutils.docterm_matrix(newsgroups['text'], 
                                 doc_names,
                                 min_df=2,
                                 token_pattern="[a-zA-Z]+")   
docterm

,a,aa,aammmaaaazzzzzziinnnnggggg,aaron,aas,ab,abandon,abandoned,abandonment,abbey,...,zip,zipping,zippy,zj,zone,zoo,zoology,zorba,zt,zv
doc0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc1053,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1054,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1055,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1056,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Notice that we cut the number of features in the space to just about half of the  original features and the features look more like words.

## Stop Words

Stop word filtering is a way to reduce dimensionality of the feature space by removing words from the document that do not add to content/concept of the document.  Words like 'its', 'an', 'the', 'for', 'that', etc. are so common in each document that they do not any value during an analysis.  We will filter them out.

In [12]:
# build vector model with stop_words="english"
# we remove all common English stop words (e.g., "the", "is", "and")
docterm = dsutils.docterm_matrix(newsgroups['text'], 
                                 doc_names,
                                 min_df=2,
                                 token_pattern="[a-zA-Z]+",
                                 stop_words="english")   
docterm

,aa,aammmaaaazzzzzziinnnnggggg,aaron,aas,ab,abandon,abandoned,abandonment,abbey,abc,...,zip,zipping,zippy,zj,zone,zoo,zoology,zorba,zt,zv
doc0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc1053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1054,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1055,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Notice that stop word filtering reduced the dimensionality by another 300 dimensions.

## Stemming

The first few coordinates are now:

['aa', 'aammmaaaazzzzzziinnnnggggg', 'aaron', 'aas', 'ab', **'abandon'**, **'abandoned'**, **'abandonment'**, 'abbey', 'abc']

Here, we see one more issue, three different shapes of the same root word, in this case **abandon**.

> Solution: Stemming!


In linguistic morphology and information retrieval, stemming is the process of reducing inflected (or sometimes derived) words to their word stem or root form.

A stemmer for English, for example, should identify the string "cats" (and possibly "catlike", "catty" etc.) as based on the root "cat", and "stems", "stemmer", "stemming", "stemmed" as based on "stem".

A stemming algorithm reduces the words "fishing", "fished", and "fisher" to the root word, "fish".

The most popular stemming algorithm:

> The [Porter Stemmer](https://en.wikipedia.org/wiki/Stemming)

This is the stemming algorithm our 'docterm_matrix' function uses.


Let's build a docterm matrix using stemming.

In [13]:
# build vector model with stemming
docterm = dsutils.docterm_matrix(newsgroups['text'], 
                                 doc_names,
                                 min_df=2,
                                 token_pattern="[a-zA-Z]+",
                                 stop_words="english",
                                 stem=True)   
docterm

,aa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abbey,abc,abdkw,abett,abid,...,zimmer,zip,zippi,zj,zone,zoo,zoolog,zorba,zt,zv
doc0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc1053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1054,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1055,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc1056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Notice that 'abandon', 'abandoned', and 'abandonment' have been mapped into the word 'abandon'.  Also notice that our final dimensionality for our feature space is now around 8,000+ features compared to the original 23,000 features.  That
is a more than 50% drop in the number of features.  This also means that we will
save 50% of effort during any kind of analysis on this document collection.

# Doc Similarity in High-Dimensional Spaces

In [14]:
distances = pandas.DataFrame(data=euclidean_distances(docterm),
                                index=doc_names,
                                columns=doc_names)
distances

,doc0,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,...,doc1048,doc1049,doc1050,doc1051,doc1052,doc1053,doc1054,doc1055,doc1056,doc1057
doc0,0.000000,12.041595,12.727922,13.711309,11.489125,14.142136,15.198684,11.532563,12.449900,11.489125,...,14.662878,11.313708,13.601471,26.570661,12.529964,11.832160,15.842980,12.649111,11.000000,13.711309
doc1,12.041595,0.000000,12.449900,13.747727,11.269428,13.747727,14.491377,10.677078,11.135529,10.816654,...,13.711309,10.344080,12.649111,26.589472,12.000000,11.445523,15.099669,11.789826,10.862780,12.767145
doc2,12.727922,12.449900,0.000000,13.928388,11.575837,14.352700,14.525839,11.532563,12.688578,11.661904,...,14.456832,11.401754,13.379088,26.570661,12.609520,12.000000,15.842980,12.649111,11.618950,13.564660
doc3,13.711309,13.747727,13.928388,0.000000,12.961481,15.099669,15.652476,12.767145,13.601471,13.038405,...,15.588457,12.727922,14.594520,26.944387,13.674794,12.727922,16.462078,13.856406,12.845233,14.764823
doc4,11.489125,11.269428,11.575837,12.961481,0.000000,13.490738,14.106736,10.148892,11.445523,10.488088,...,13.601471,10.099505,12.609520,26.267851,11.357817,10.862780,15.000000,11.401754,10.344080,12.489996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc1053,11.832160,11.445523,12.000000,12.727922,10.862780,13.416408,14.317821,10.246951,11.445523,10.583005,...,13.674794,10.392305,12.529964,26.457513,11.357817,0.000000,14.933185,11.575837,10.630146,12.884099
doc1054,15.842980,15.099669,15.842980,16.462078,15.000000,16.703293,17.435596,14.491377,15.362291,14.730920,...,17.204651,14.456832,16.186414,27.658633,15.427249,14.933185,0.000000,15.716234,14.899664,16.155494
doc1055,12.649111,11.789826,12.649111,13.856406,11.401754,13.784049,14.594520,10.816654,11.874342,11.045361,...,9.848858,10.392305,13.076697,26.683328,12.124356,11.575837,15.716234,0.000000,11.445523,13.114877
doc1056,11.000000,10.862780,11.618950,12.845233,10.344080,13.379088,14.212670,10.000000,11.313708,10.246951,...,13.784049,9.643651,12.083046,26.210685,11.045361,10.630146,14.899664,11.445523,0.000000,12.288206


## Find out which stories are most similar

In [15]:
# map the major diagonal into FLOAT_MAX
# We know that a doc is always identical to itself.
for i in range(distances.shape[0]):
   distances.iloc[i,i] = sys.float_info.max

In [16]:
# find the column with the minimal value
cix = distances.min(axis=0)\
               .idxmin()
cix

'doc337'

In [17]:
# find the row with the minimal value given the first document
rix= distances[cix].idxmin()
rix

'doc1037'

In [18]:
# these two news stories are most similar
f"Distance between stories: {distances.loc[rix, cix]}"

'Distance between stories: 0.0'

**Observation**: A distance of 0 means the two docs are identical!

In [19]:
# text of the first message
print(newsgroups['text'].loc[rix])

From: tkelso@afit.af.mil (TS Kelso)
Subject: Two-Line Orbital Element Set:  Space Shuttle
Keywords: Space Shuttle, Orbital Elements, Keplerian
Nntp-Posting-Host: scgraph.afit.af.mil
Organization: Air Force Institute of Technology
Lines: 21

The most current orbital elements from the NORAD two-line element sets are
carried on the Celestial BBS, (513) 427-0674, and are updated daily (when
possible).  Documentation and tracking software are also available on this
system.  As a service to the satellite user community, the most current
elements for the current shuttle mission are provided below.  The Celestial
BBS may be accessed 24 hours/day at 300, 1200, 2400, 4800, or 9600 bps using
8 data bits, 1 stop bit, no parity.

Element sets (also updated daily), shuttle elements, and some documentation
and software are also available via anonymous ftp from archive.afit.af.mil
(129.92.1.66) in the directory pub/space.

STS 56     
1 22621U 93 23  A 93105.58333333  .00090711  00000-0  25599-3 0   2

In [20]:
# text of the second message
print(newsgroups['text'].loc[cix])

From: tkelso@afit.af.mil (TS Kelso)
Subject: Two-Line Orbital Element Set:  Space Shuttle
Keywords: Space Shuttle, Orbital Elements, Keplerian
Nntp-Posting-Host: scgraph.afit.af.mil
Organization: Air Force Institute of Technology
Lines: 21

The most current orbital elements from the NORAD two-line element sets are
carried on the Celestial BBS, (513) 427-0674, and are updated daily (when
possible).  Documentation and tracking software are also available on this
system.  As a service to the satellite user community, the most current
elements for the current shuttle mission are provided below.  The Celestial
BBS may be accessed 24 hours/day at 300, 1200, 2400, 4800, or 9600 bps using
8 data bits, 1 stop bit, no parity.

Element sets (also updated daily), shuttle elements, and some documentation
and software are also available via anonymous ftp from archive.afit.af.mil
(129.92.1.66) in the directory pub/space.

STS 56     
1 22621U 93 23  A 93105.06179397  .00044513  00000-0  12649-3 0   2

> It is a repeated article in the dataset!